<a href="https://colab.research.google.com/github/codeplay9800/Sound-Classification-with-Machine-Learning/blob/main/STAT471_FINAL_PROJECT_SUMMARY_NOTEBOOK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup

This Jupyter notebook loads the raw amplitude and Mel spectrogram data files as numpy arrays.

Download the data files [here](https://console.cloud.google.com/storage/browser/cs181_practical_data).  This notebook assumes that the data files as located in the same directory.

In [1]:
!pip install dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 3.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import glob
import os
import sys
import matplotlib.pyplot as plt
from numpy.linalg import *
np.random.seed(42)  # don't change this line
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits
import seaborn as sns
%matplotlib inline
import dill
import base64

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Course Folders/STAT4710 Data Mining/stat471 final project

[Errno 2] No such file or directory: '/content/drive/MyDrive/Course Folders/STAT4710 Data Mining/stat471 final project'
/content


# 1. Data collection & EDA

### 1.1 Load raw amplitude data.

In [5]:
# Check that loading in amp data works correctly
test_fold = glob.glob('./data/Amp_Data/*_X.npy')
print(len(test_fold))
x_load = np.load(test_fold[0])
X_train = np.empty(0)
X_train = np.append(X_train, x_load.flatten())
X_train = X_train.reshape(x_load.shape)
if np.array_equal(x_load, X_train, equal_nan= True):
  print("same")
else:
  print("not same")

0


IndexError: list index out of range

In [ ]:
#Load Train Data

test_fold_X = glob.glob('./data/Amp_Data/*_X.npy')
print(len(test_fold_X))

X_amp_train = np.empty(0)

num_rows_x = 0
num_cols_x = 44100
for i in range(len(test_fold_X)):
  if test_fold_X[i] == './data/Amp_Data/fold10_amp_X.npy':
    continue
  x_load = np.load(test_fold_X[i])
  X_amp_train = np.append(X_amp_train, x_load.flatten())
  num_rows_x += x_load.shape[0]

print(num_rows_x)
X_amp_train = X_amp_train.reshape( num_rows_x, num_cols_x )

test_fold_Y = glob.glob('./data/Amp_Data/*_Y.npy')
print(len(test_fold_Y))

y_amp_train = np.empty(0)

num_rows_y = 0
for i in range(len(test_fold_Y)):
  if test_fold_Y[i] == './data/Amp_Data/fold10_amp_Y.npy':
    continue
  y_load = np.load(test_fold_Y[i])
  y_amp_train = np.append(y_amp_train, y_load.flatten())
  num_rows_y += y_load.shape[0]

print(y_amp_train.shape)

assert num_rows_x == num_rows_y

In [ ]:
print(test_fold_X)
print(test_fold_Y)
# fold10 (used for testing) is at idx 3

In [ ]:
X_amp_train.shape

In [ ]:
# Load Test data
test_fold_X = glob.glob('./data/Amp_Data/*_X.npy')
print(len(test_fold_X))

X_amp_test = np.load(test_fold_X[3])

test_fold_Y = glob.glob('./data/Amp_Data/*_Y.npy')
print(len(test_fold_Y))

y_amp_test = np.load(test_fold_Y[3]).flatten()
print(y_amp_test.shape)

assert X_amp_test.shape[0] == y_amp_test.shape[0]

In [ ]:
X_amp_test.shape

## 1.2 Load spectogram data

In [ ]:
#Load Spectogram Train Data

train_fold_X_spec = glob.glob('./data/Fold_Spec_New/*_X.npy')
print(len(train_fold_X_spec))

X_spec_train = np.empty(0)

num_rows = 128
num_cols = 87
for i in range(len(train_fold_X_spec)):
  if train_fold_X_spec[i] == './data/Fold_Spec_New/fold10_spec_X.npy':
    print("Found last fold")
    continue
  x_load = np.load(train_fold_X_spec[i])
  print(x_load.shape)
  X_spec_train = np.append(X_spec_train, x_load.flatten())
  del x_load # free memory

X_spec_train = X_spec_train.reshape( -1 , num_rows *  num_cols)

train_fold_Y_Spec = glob.glob('./data/Fold_Spec_New/*_Y.npy')
print(len(train_fold_Y_Spec))

y_spec_train = np.empty(0)

for i in range(len(train_fold_Y_Spec)):
  if train_fold_Y_Spec[i] == './data/Fold_Spec_New/fold10_spec_Y.npy':
    continue
  y_load = np.load(train_fold_Y_Spec[i])
  y_spec_train = np.append(y_spec_train, y_load.flatten())
  del y_load # free memory

print(y_spec_train.shape)

assert X_spec_train.shape[0] == y_spec_train.shape[0]

In [ ]:
# Load Spectogram Test data
test_fold_X_spec = './data/Fold_Spec_New/fold10_spec_X.npy'

X_spec_test = np.load(test_fold_X_spec)
X_spec_test = X_spec_test.reshape( -1 , num_rows *  num_cols)

test_fold_Y_spec = './data/Fold_Spec_New/fold10_spec_Y.npy'

y_spec_test = np.load(test_fold_Y_spec).flatten()
print(y_spec_test.shape)

assert X_spec_test.shape[0] == y_spec_test.shape[0]

In [ ]:
# plot spectogram data
import librosa
import librosa.display
Load_Data = './data/Fold_Spec_New/fold8_spec_X.npy'
mel_spec = np.load(Load_Data)
librosa.display.specshow(mel_spec[1].reshape(128, 87))

# 2. Modeling

## 2.1 Baseline model (Logistic Regression)

### 2.1.1 Trained on amplitude data

In [ ]:
# Train a Logisc Regression Classifier

logistic_classifier = LogisticRegression(random_state = 0, max_iter = 100000)

X_amp_train_shortened = X_amp_train[:2000]
Y_amp_train_shortened = y_amp_train[:2000]
logistic_classifier.fit(X_amp_train_shortened, Y_amp_train_shortened)

In [ ]:
print(logistic_classifier.score(X_amp_train_shortened, Y_amp_train_shortened))

In [ ]:
X_amp_test_shortened = X_amp_test[:100]
Y_amp_test_shortened = y_amp_test[:100]
logistic_classifier.score(X_amp_test_shortened, Y_amp_test_shortened)

### 2.1.2 Trained on spectogram data

In [ ]:
logistic_classifier_spec = LogisticRegression(random_state = 0, max_iter = 1000)

# X_amp_train_shortened = X_spec_train[:2000]
# Y_amp_train_shortened = y_spec_train[:2000]
logistic_classifier_spec.fit(X_spec_train[:2000], y_spec_train[:2000])

In [ ]:
# getting score on train data
print(X_spec_train[2100].shape)
logistic_classifier_spec.score(X_spec_train[2000:2060], y_spec_train[2000:2060] )

## 2.2 Principal Component Analysis

We will now run PCA on the feature set of the WAV files (currently 44k dimensions) in order to subsequently run KNN without introducing the effects of the curse of dimensionality.

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(X_amp_train_shortened)
X_amp_train_scaled_shortened = scaler.transform(X_amp_train_shortened)

In [ ]:
# Use the same scalar to scale the corresponding testing data
X_amp_test_scaled_shortened = scaler.transform(X_amp_test_shortened)

In [ ]:
import sklearn.decomposition as skdecomp

pca = skdecomp.PCA(n_components=2)
pca.fit(X_amp_train_scaled_shortened)
X_amp_train_pca = pca.transform(X_amp_train_scaled_shortened)

In [ ]:
import matplotlib.pyplot as plt


plt.scatter(X_amp_train_pca[:, 0], X_amp_train_pca[:, 1])
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.show()

In [ ]:
pca = skdecomp.PCA(n_components=3)
pca.fit(X_amp_train_scaled_shortened)
X_amp_train_pca = pca.transform(X_amp_train_scaled_shortened)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.xlim(-5, 5)
plt.ylim(-5, 5)

ax.scatter(X_amp_train_pca[:, 0], X_amp_train_pca[:, 1], X_amp_train_pca[:, 2])

Now, we will proceed with calculating the covariance matrix, and visualize how increasing the number of features explains the variance within the data.

In [ ]:
scaler_spec = StandardScaler()
scaler_spec.fit(X_spec_train)
X_spec_train = scaler_spec.transform(X_spec_train)
X_spec_test = scaler_spec.transform(X_spec_test)

In [ ]:
covar_matrix = skdecomp.PCA(n_components = 3000)
covar_matrix.fit(X_spec_train)


In [ ]:
variance = covar_matrix.explained_variance_ratio_

var = np.cumsum(np.round(variance, decimals=3)*100)

In [ ]:
print(var)

plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.style.context('seaborn-whitegrid')


plt.plot(var)

In [ ]:
for i in range(10, 100):
  print(f"numFeatures = {i}, % variance explained = {var[i]}")
  # ~55 features is a good choice for numFeatures to include

In [ ]:
pca_final = skdecomp.PCA(n_components=100)
pca_final.fit(X_spec_train)
X_spec_train_final_pca = pca_final.transform(X_spec_train)

final_variance = pca_final.explained_variance_ratio_

final_var = np.cumsum(np.round(final_variance, decimals=3)*100)
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.style.context('seaborn-whitegrid')


plt.plot(final_var)
# ok now we have reduced dimension data (60-D)

In [ ]:
print(X_spec_train_final_pca.shape)

### 2.2.1 Logistic classifier on spec PCA features

In [ ]:
# now we do some logistic regression on smaller data
lg_clf = LogisticRegression(random_state = 0, max_iter = 1000)
lg_clf.fit(X_spec_train_final_pca, y_spec_train)

In [ ]:
# pca transform the test data
X_spec_test_final_pca = pca_final.transform(X_spec_test)

In [ ]:
# Test lg classifier on train data
lg_clf.score(X_spec_train_final_pca[100:110], y_spec_train[100:110])

In [ ]:
lg_clf.score(X_spec_test_final_pca, y_spec_test)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

y_pred = lg_clf.predict(X_spec_test_final_pca)

accuracy = accuracy_score(y_spec_test, y_pred)
print(f"Accuracy: {accuracy}")

recall = recall_score(y_spec_test, y_pred, average = "macro")
print(f"Recall: {recall}")

precision = precision_score(y_spec_test, y_pred, average = "macro")
print(f"Precision: {precision}")

f1 = f1_score(y_spec_test, y_pred, average = "macro")
print(f"F1 score: {f1}")

## 2.3 Random Forests classifier

The next classifier model we tested was Random Forests, which takes an ensemble of decision trees, each of which outputs a predicted class label, and takes the majority vote as the random forests classifier prediction.


### 2.3.1 on amp data (reduced dimensionality)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators = 100, random_state = 0)
rf_model.fit(X_amp_train_shortened, Y_amp_train_shortened)

In [ ]:
train_score = rf_model.score(X_amp_train_shortened, Y_amp_train_shortened)
test_score = rf_model.score(X_amp_test_shortened, Y_amp_test_shortened)
print(f"Score on amp train data: {train_score}, Score on amp test data: {test_score}")

### 2.3.2 on amp data (full)

In [ ]:
rf_model2 = RandomForestClassifier(n_estimators = 100, random_state = 1)
rf_model2.fit(X_amp_train, y_amp_train)

In [ ]:
train_score = rf_model.score(X_amp_train, y_amp_train)
test_score = rf_model.score(X_amp_test, y_amp_test)
print(f"Score on full amp train data: {train_score}")
print(f"Score on full amp test data: {test_score}")

### 2.3.3 on spec data

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model_spec = RandomForestClassifier(n_estimators = 100, random_state = 0)
rf_model_spec.fit(X_spec_train, y_spec_train)

In [ ]:
spec_train_score = rf_model_spec.score(X_spec_train, y_spec_train)
spec_test_score = rf_model_spec.score(X_spec_test, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

### 2.3.4 on spec data (PCA-transformed)

As expected, random forests performed well and had results more generalizable to the test set compared to other classifiers, as it is more robust to overfitting on the training set due to relying on a committee of relatively-uncorrelated individual decision trees.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model_spec_PCA = RandomForestClassifier(n_estimators = 100, random_state = 0)
rf_model_spec_PCA.fit(X_spec_train_final_pca, y_spec_train)

In [ ]:
spec_train_score = rf_model_spec_PCA.score(X_spec_train_final_pca, y_spec_train)
spec_test_score = rf_model_spec_PCA.score(X_spec_test_final_pca, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = rf_model_spec_PCA.predict(X_spec_test_final_pca)
cm = confusion_matrix(y_spec_test, y_pred)

class_labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'enginge_idling', 'gun_shot', 'jackhammer', 'siren','street_music']

plt.figure(figsize = (10, 7))
sns.heatmap(cm, annot = True, fmt = "d", xticklabels = class_labels, yticklabels = class_labels)
plt.title("Confusion matrix")
plt.ylabel("Actual label")
plt.xlabel("Predicted label")
plt.show()

From the confusion matrix above, we see that Random Forests performed relatively well in categorizing most of the audio sample classes, but struggled
for some specific classes such as ’drilling’ and ’siren’, which had several misclassified points. We believe this is likely explained by the fact that audio samples in some of the classes like ’drilling’ are of similar ampli-
tude to the other classes, making it hard for the classifier models to distinguish.

In [ ]:
y_pred = rf_model_spec_PCA.predict(X_spec_test_final_pca)

accuracy = accuracy_score(y_spec_test, y_pred)
print(f"Accuracy: {accuracy}")

recall = recall_score(y_spec_test, y_pred, average = "macro")
print(f"Recall: {recall}")

precision = precision_score(y_spec_test, y_pred, average = "macro")
print(f"Precision: {precision}")

f1 = f1_score(y_spec_test, y_pred, average = "macro")
print(f"F1 score: {f1}")

Checking the accuracy, recall, and F1 score above, we see that we observed that the Random Forests classifier performed the best out of the traditional
ML models, exhibiting a test accuracy of 67.14% and high scores in the range of 60 − 70% for F1, precision, recall.

In [ ]:
class_accuracies = np.diag(cm) / np.sum(cm, axis = 1)
sorted_class_accuracies = np.sort(class_accuracies)
sorted_indices = np.argsort(class_accuracies)
sorted_class_labels = np.array(class_labels)[sorted_indices]

plt.figure(figsize = (10, 7))
plt.barh(sorted_class_labels, sorted_class_accuracies)
plt.title("Class accuracies")
plt.xlabel("Accuracy")
plt.ylabel("Class label")

for i, v in enumerate(sorted_class_accuracies):
    plt.text(v, i, str(round(v, 2)), color='blue', fontweight='bold')

plt.show()

As shown in the individual class accuracies graph above, distinguishable classes like ’gunshot’ tended to have very high class accuracies (91%), whereas the low-distinguishability classes like 'drilling' and 'siren' had lower accuracies (46% and 52%) that drove down the random forests classifier's overall accuracy score.

## 2.4 Naive Bayes classifier

### 2.4.1 on amp data

In [ ]:
num_classes = len(np.unique(Y_amp_test_shortened))
print(num_classes)

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_model.fit(X_amp_train_shortened, Y_amp_train_shortened)

In [ ]:
train_score = nb_model.score(X_amp_train_shortened, Y_amp_train_shortened)
test_score = nb_model.score(X_amp_test_shortened, Y_amp_test_shortened)
print(f"Score on shortened amp train data: {train_score}, Score on shortened amp test data: {test_score}")

### 2.4.2 on spec data

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_model_spec = GaussianNB()
nb_model_spec.fit(X_spec_train, y_spec_train)

In [ ]:
spec_train_score = nb_model_spec.score(X_spec_train, y_spec_train)
spec_test_score = nb_model_spec.score(X_spec_test, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

### 2.4.3 on spec data (PCA-transformed)

In [ ]:
nb_model_spec_PCA = GaussianNB()
nb_model_spec_PCA.fit(X_spec_train_final_pca, y_spec_train)

In [ ]:
spec_train_score = nb_model_spec_PCA.score(X_spec_train_final_pca, y_spec_train)
spec_test_score = nb_model_spec_PCA.score(X_spec_test_final_pca, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

In [ ]:
y_pred = nb_model_spec_PCA.predict(X_spec_test_final_pca)

accuracy = accuracy_score(y_spec_test, y_pred)
print(f"Accuracy: {accuracy}")

recall = recall_score(y_spec_test, y_pred, average = "macro")
print(f"Recall: {recall}")

precision = precision_score(y_spec_test, y_pred, average = "macro")
print(f"Precision: {precision}")

f1 = f1_score(y_spec_test, y_pred, average = "macro")
print(f"F1 score: {f1}")

## 2.5 K-Nearest Neighbor (kNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model_spec_PCA = KNeighborsClassifier(n_neighbors = 5)
knn_model_spec_PCA.fit(X_spec_train_final_pca, y_spec_train)

spec_train_score = knn_model_spec_PCA.score(X_spec_train_final_pca, y_spec_train)
spec_test_score = knn_model_spec_PCA.score(X_spec_test_final_pca, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

In [ ]:
y_pred = knn_model_spec_PCA.predict(X_spec_test_final_pca)

accuracy = accuracy_score(y_spec_test, y_pred)
print(f"Accuracy: {accuracy}")

recall = recall_score(y_spec_test, y_pred, average = "macro")
print(f"Recall: {recall}")

precision = precision_score(y_spec_test, y_pred, average = "macro")
print(f"Precision: {precision}")

f1 = f1_score(y_spec_test, y_pred, average = "macro")
print(f"F1 score: {f1}")

Visualization of the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = knn_model_spec_PCA.predict(X_spec_test_final_pca)
cm = confusion_matrix(y_spec_test, y_pred)

class_labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'enginge_idling', 'gun_shot', 'jackhammer', 'siren','street_music']

plt.figure(figsize = (10, 7))
sns.heatmap(cm, annot = True, fmt = "d", xticklabels = class_labels, yticklabels = class_labels)
plt.title("Confusion matrix")
plt.ylabel("Actual label")
plt.xlabel("Predicted label")
plt.show()

## 2.6 Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC

svm_model_spec_PCA = SVC(kernel = 'rbf', random_state = 0)
svm_model_spec_PCA.fit(X_spec_train_final_pca, y_spec_train)

spec_train_score = svm_model_spec_PCA.score(X_spec_train_final_pca, y_spec_train)
spec_test_score = svm_model_spec_PCA.score(X_spec_test_final_pca, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

In [ ]:
y_pred = svm_model_spec_PCA.predict(X_spec_test_final_pca)

accuracy = accuracy_score(y_spec_test, y_pred)
print(f"Accuracy: {accuracy}")

recall = recall_score(y_spec_test, y_pred, average = "macro")
print(f"Recall: {recall}")

precision = precision_score(y_spec_test, y_pred, average = "macro")
print(f"Precision: {precision}")

f1 = f1_score(y_spec_test, y_pred, average = "macro")
print(f"F1 score: {f1}")

### 2.6.1 Hyperparameter Tuning (on C and gamma)

**C:** adds a penalty for each misclassified data point. If C is large, SVM tries to minimize the number of misclassified examples due to the high penalty which results in a decision boundary with a smaller margin.

**Gamma:** values of gamma indicate a large similarity radius which results in more points being grouped together. For high values of gamma, the points need to be very close to each other in order to be considered in the same group (or class).

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10],
              'gamma': ['scale', 'auto'],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
grid.fit(X_spec_train_final_pca, y_spec_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)
print(grid.score(X_spec_test_final_pca, y_spec_test))

In [ ]:
svm_tuned = SVC(C=10, kernel='rbf', random_state=0)
svm_tuned.fit(X_spec_train_final_pca, y_spec_train)

spec_train_score = svm_tuned.score(X_spec_train_final_pca, y_spec_train)
spec_test_score = svm_tuned.score(X_spec_test_final_pca, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

## 2.7 XGBoost classifier (gradient-boosted decision trees)

In [ ]:
from xgboost import XGBClassifier

xgb_model_spec_PCA = XGBClassifier()
xgb_model_spec_PCA.fit(X_spec_train_final_pca, y_spec_train)

spec_train_score = xgb_model_spec_PCA.score(X_spec_train_final_pca, y_spec_train)
spec_test_score = xgb_model_spec_PCA.score(X_spec_test_final_pca, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

In [ ]:
y_pred = xgb_model_spec_PCA.predict(X_spec_test_final_pca)

accuracy = accuracy_score(y_spec_test, y_pred)
print(f"Accuracy: {accuracy}")

recall = recall_score(y_spec_test, y_pred, average = "macro")
print(f"Recall: {recall}")

precision = precision_score(y_spec_test, y_pred, average = "macro")
print(f"Precision: {precision}")

f1 = f1_score(y_spec_test, y_pred, average = "macro")
print(f"F1 score: {f1}")

# 3. Further Analysis

## 3.1 Neural networks (MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_model_spec_PCA = MLPClassifier(hidden_layer_sizes = (1000, 500, 250), max_iter = 2000)
mlp_model_spec_PCA.fit(X_spec_train_final_pca, y_spec_train)

spec_train_score = mlp_model_spec_PCA.score(X_spec_train_final_pca, y_spec_train)
spec_test_score = mlp_model_spec_PCA.score(X_spec_test_final_pca, y_spec_test)
print(f"Score on spec train data: {spec_train_score}")
print(f"Score on spec test data: {spec_test_score}")

In [ ]:
y_pred = mlp_model_spec_PCA.predict(X_spec_test_final_pca)

accuracy = accuracy_score(y_spec_test, y_pred)
print(f"Accuracy: {accuracy}")

recall = recall_score(y_spec_test, y_pred, average = "macro")
print(f"Recall: {recall}")

precision = precision_score(y_spec_test, y_pred, average = "macro")
print(f"Precision: {precision}")

f1 = f1_score(y_spec_test, y_pred, average = "macro")
print(f"F1 score: {f1}")

## 3.2 Model 1: Simple neural network (only dense layers with ReLU & softmax)

Since the class labels are currently stored as strings, we converted them to one-hot encoding for the CNN

In [ ]:
from tensorflow.keras.utils import to_categorical

y_spec_train_one_hot = to_categorical(y_spec_train)
y_spec_test_one_hot = to_categorical(y_spec_test)

In [ ]:
# X_spec_train_final_pca.shape is (7895, 60)
# X_spec_test_final_pca.shape is (837, 60)
# y_spec_train.shape is (7895,)
# y_spec_test.shape is (837,)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Flatten(),
    layers.Dense(100, activation = 'relu', input_shape = (100,)),
    layers.Dense(100, activation = 'relu'),
    layers.Dense(100, activation = 'relu'),
    layers.Dense(10, activation = 'softmax'),
])

model.compile(optimizer = 'adam',
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

model.fit(X_spec_train_final_pca, y_spec_train_one_hot, epochs = 50, validation_split=0.1, shuffle=True)

In [ ]:
model.summary()

In [ ]:
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label = 'Training loss')
plt.plot(epochs, val_loss, label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, label = 'Training accuracy')
plt.plot(epochs, val_acc, label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_spec_test_final_pca, y_spec_test_one_hot, verbose = 2)

print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

y_pred = model.predict(X_spec_test_final_pca)
y_pred = np.argmax(y_pred, axis = 1)
y_actual = np.argmax(y_spec_test_one_hot, axis = 1)

print(f"Accuracy: {accuracy_score(y_actual, y_pred)}")
print(f"Recall: {recall_score(y_actual, y_pred, average = 'macro')}")
print(f"Precision: {precision_score(y_actual, y_pred, average = 'macro')}")
print(f"F1 score: {f1_score(y_actual, y_pred, average = 'macro')}")


## 3.3 Model 2: Convolutional Neural Network (CNN)

In [ ]:
model2 = keras.Sequential([
    layers.Conv1D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = (100, 1)),
    layers.MaxPooling1D(pool_size = 2),
    layers.Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    layers.MaxPooling1D(pool_size = 2),
    layers.Flatten(),
    layers.Dense(100, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])

model2.compile(optimizer = 'adam',
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

model2.fit(X_spec_train_final_pca, y_spec_train_one_hot, epochs = 60, validation_split=0.2, shuffle=True)

In [ ]:
loss = model2.history.history['loss']
val_loss = model2.history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label = 'Training loss')
plt.plot(epochs, val_loss, label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
acc = model2.history.history['accuracy']
val_acc = model2.history.history['val_accuracy']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, label = 'Training accuracy')
plt.plot(epochs, val_acc, label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
model2.summary()

In [ ]:
y_pred = model2.predict(X_spec_test_final_pca)
y_pred = np.argmax(y_pred, axis = 1)
y_actual = np.argmax(y_spec_test_one_hot, axis = 1)

print(f"Accuracy: {accuracy_score(y_actual, y_pred)}")
print(f"Recall: {recall_score(y_actual, y_pred, average = 'macro')}")
print(f"Precision: {precision_score(y_actual, y_pred, average = 'macro')}")
print(f"F1 score: {f1_score(y_actual, y_pred, average = 'macro')}")

In [ ]:
test_loss, test_acc = model2.evaluate(X_spec_test_final_pca, y_spec_test_one_hot, verbose = 2)

print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

## 3.4 Model 3: CNN with dropout & regularization (to reduce overfitting)

In [ ]:
from tensorflow.keras import regularizers

model3 = keras.Sequential([
    layers.Conv1D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = (100, 1)),
    layers.MaxPooling1D(pool_size = 2),
    layers.Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    layers.MaxPooling1D(pool_size = 2),
    layers.Flatten(),
    layers.Dense(100, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(10, activation = 'softmax')
])

model3.compile(optimizer = 'adam',
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

hist = model3.fit(X_spec_train_final_pca, y_spec_train_one_hot, epochs = 50, validation_split=0.1, shuffle=True)

In [ ]:
model3.summary()

In [ ]:
test_loss, test_acc = model3.evaluate(X_spec_test_final_pca, y_spec_test_one_hot, verbose = 2)

print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

In [ ]:
y_pred = model3.predict(X_spec_test_final_pca)
y_pred = np.argmax(y_pred, axis = 1)
y_actual = np.argmax(y_spec_test_one_hot, axis = 1)

print(f"Accuracy: {accuracy_score(y_actual, y_pred)}")
print(f"Recall: {recall_score(y_actual, y_pred, average = 'macro')}")
print(f"Precision: {precision_score(y_actual, y_pred, average = 'macro')}")
print(f"F1 score: {f1_score(y_actual, y_pred, average = 'macro')}")

In [ ]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label = 'Training loss')
plt.plot(epochs, val_loss, label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, label = 'Training accuracy')
plt.plot(epochs, val_acc, label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

## 3.4b Audio Classification Specific CNN

In [ ]:
X_spec_train_full = X_spec_train.reshape(7895, 128, -1)

In [ ]:
X_spec_test_full = X_spec_test.reshape(-1, 128, 87)

In [ ]:
import pickle
pickle.dump(X_spec_train_full, open('X_spec_train_full.pkl', 'wb'))
pickle.dump(y_spec_train_one_hot, open('y_spec_train_one_hot.pkl', 'wb'))
pickle.dump(X_spec_test_full, open('X_spec_test_full.pkl', 'wb'))
pickle.dump(y_spec_test_one_hot, open('y_spec_test_one_hot.pkl', 'wb'))

These bigger CNNs are created/trained in a separate Colab notebook.

## 3.5 Comparison to State-of-the-art SpeechBrain model (ECAPA-TDNN)

In [ ]:
!pip install speechbrain

In [ ]:
from speechbrain.pretrained import EncoderClassifier

model = EncoderClassifier.from_hparams(source = "speechbrain/urbansound8k_ecapa", savedir = "pretrained_models/urbansound8k_ecapa")
model.modules

This would involve directly feeding in an array of WAV files. This is too computationally expensive for Colab, but you can try to pass in individual WAV files and call `classify_batch` to classify them!

# 4. Conclusion

Visualization of confusion matrix and individual class accuracies for random forests, our best model (excluding neural net models).

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = rf_model_spec_PCA.predict(X_spec_test_final_pca)
cm = confusion_matrix(y_spec_test, y_pred)

class_labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'enginge_idling', 'gun_shot', 'jackhammer', 'siren','street_music']

plt.figure(figsize = (10, 7))
sns.heatmap(cm, annot = True, fmt = "d", xticklabels = class_labels, yticklabels = class_labels)
plt.title("Confusion matrix")
plt.ylabel("Actual label")
plt.xlabel("Predicted label")
plt.show()

In [ ]:
class_accuracies = np.diag(cm) / np.sum(cm, axis = 1)
sorted_class_accuracies = np.sort(class_accuracies)
sorted_indices = np.argsort(class_accuracies)
sorted_class_labels = np.array(class_labels)[sorted_indices]

plt.figure(figsize = (10, 7))
plt.barh(sorted_class_labels, sorted_class_accuracies)
plt.title("Class accuracies")
plt.xlabel("Accuracy")
plt.ylabel("Class label")

for i, v in enumerate(sorted_class_accuracies):
    plt.text(v, i, str(round(v, 2)), color='blue', fontweight='bold')

plt.show()

In [ ]:
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# %%capture
!jupyter nbconvert --to html /content/drive/MyDrive/"Course Folders"/"STAT4710 Data Mining"/"stat471 final project"/"STAT471 FINAL PROJECT SUMMARY NOTEBOOK.ipynb"